# Команда "Друзья, цели, победы"
## Спеллчекер с проверкой по частотным биграммам

In [8]:
import operator
import nltk
from nltk import bigrams
import re
from collections import Counter

Cоздаем словарь из списка биграмм OpenCorpora, где ключ -- биграмма, а значение -- ipm.

In [9]:
file = open('bigrams.cyrB.txt', 'r', encoding = 'utf-8')
bigrams1 = file.readlines()
our_bigrams = []
for bigram in bigrams1:
    our_bigrams.append(bigram.split('\t'))
for bigram in our_bigrams:
    bigram[2] = int(bigram[1].strip('\n'))
    del bigram[1]
bigrams_ipm = {b[0]:b[1] for b in our_bigrams}
bigrams_ipm = {tuple(k.split(' ')):int(v) for k,v in bigrams_ipm.items()}

file = open('unigrams.txt', 'r', encoding = 'utf-8')
words1 = file.readlines()
our_words = []
for word in words1:
    our_words.append(word.split('\t'))
for word in our_words:
    del word[2]
    word[0] = word[0].lower()
words_ipm = {w[0]:int(w[1]) for w in our_words}



Функция Питера Норвига

In [10]:
def words(text): return re.findall(r'\w+', text.lower())

WORDS1 = dict(Counter(words(open('dict.opcorpora1.txt').read())))

WORDS = {**WORDS1, **words_ipm}

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    if word in WORDS:
        return WORDS[word] / N
    else:
        return 0


def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

Проверка по частотным биграммам

In [11]:
def preprocessing(string):
    string = string.lower()
    string = string.strip()
    #string = ' '.join(re.findall('[а-яё|-]+', string))
    string = ' '.join(re.findall('[\w-]+', string))
    return string

def mistake_check(string):
    #функция находит ошибочные слова в предложении
    mistakes = []
    string = string.split()
    for word in string:
        if word not in WORDS:
            mistakes.append(word)
    return list(set(mistakes))

def generate_bigrams(string):
    #функция генерирует биграммы 
    return [gram for gram in bigrams(string.split())]

def enumerate_bigrams(string):
    return enumerate(generate_bigrams(string))

def known_bigrams(grams):
    #функция проверяет, какие биграммы из полученных находятся в словаре биграмм, 
    # понадобится позже, чтобы проверять биграммы-кандидаты
    return list(set(gram for gram in grams if gram in bigrams_ipm.keys()))
    #return set(w for w in words if w in WORDS)

def wrong_bigrams(string):
    #выдает сет биграмм с ошибочными словами
    wrong_bigrams = []
    mistakes = mistake_check(string)
    bbigrams = generate_bigrams(string)
    for gram in bbigrams:
        for word in gram:
            if word in mistakes:
                wrong_bigrams.append(gram)
    return list(set(wrong_bigrams))

def possible_bigrams(string):
    a = wrong_bigrams(string)
    possible_bigrams = []
    for w1,w2 in a:
        possible_bigrams_one = []
        if w1 in mistake_check(string) and w2 not in mistake_check(string):
            for word in list(candidates(w1)):
                empty_bigram = (word, w2)
                possible_bigrams_one.append(empty_bigram)
                empty_bigram = ((), ())
        elif w2 in mistake_check(string) and w1 not in mistake_check(string):
            for word in list(candidates(w2)):
                empty_bigram = (w1, word)
                possible_bigrams_one.append(empty_bigram)
                empty_bigram = ((), ())
        elif w1 and w2 in mistake_check(string):
            for word in list(candidates(w1)):
                for word2 in list(candidates(w2)):
                    empty_bigram = (word, word2)
                    possible_bigrams_one.append(empty_bigram)
                    empty_bigram = ((), ())
        possible_bigrams.append(possible_bigrams_one)
    return dict(zip(a,possible_bigrams))

def likely_bigrams(string):
    d = possible_bigrams(string)
    for key in d:
        if known_bigrams(d[key]):
            d[key] = known_bigrams(d[key])
        else:
            pass
    return d


def choose_bigram(string):
    d = likely_bigrams(string)
    e = {}
    for key in d.keys():
        if d[key] != []:
            for item in d[key]:
                e[item] = bigrams_ipm.get(item)
                if e[item] == None:
                    e[item] = 0
            d[key] = max(e.items(), key=operator.itemgetter(1))[0]
            e = {}
        else: d[key] = 'NO'
    return d




def replace_bigram(string):
    d = choose_bigram(string)
    a = generate_bigrams(string)
    for i, gram in enumerate(a):
        if gram in d.keys():
            if d[gram] != 'NO':
                a[i] = d[gram]
    return (a)

def restore_sentence(bigrams):
    sentence = []
    correct_sentence = []
    for w1,w2 in bigrams:
        sentence.append(w1)
        sentence.append(w2)
    if sentence[0] not in WORDS:
        correct_sentence.append(correction(sentence[0]))
    else:
        correct_sentence.append(sentence[0])
    for i in range(1, len(sentence)-1, 2):
        if sentence[i] in WORDS and sentence[i+1] not in WORDS:
            correct_sentence.append(sentence[i])
        elif sentence[i] not in WORDS and sentence[i+1] in WORDS:
            correct_sentence.append(sentence[i+1])
        elif sentence[i] == sentence[i+1]:
            correct_sentence.append(sentence[i])
        elif sentence[i] in WORDS and sentence[i+1] in WORDS:
            correct_sentence.append(max(set([sentence[i], sentence[i+1]]), key=P))
    if sentence[-1] not in WORDS:
        correct_sentence.append(correction(sentence[-1]))
    else:
        correct_sentence.append(sentence[-1])
    correct_sentence = ' '.join(correct_sentence)
    return correct_sentence

Финал

In [12]:
def correct_sentence(string):
    string = preprocessing(string)
    string = replace_bigram(string)
    string = restore_sentence(string)
    string = string.split()
    string = ' '.join(string)
    return string

Что получилось?

In [19]:
ex = 'Критеки ниандертальсково искуства утверждают, что никакого такого исскусства нет'
print('Correct sentence:\n', correct_sentence(ex))

Correct sentence:
 притеки неандертальского искусства утверждают что никакого такого искусства нет


Проверим evaluation-скриптом

In [22]:
with open('test_new.txt') as f:
    lines = f.readlines()
clean_lines = []
for line in lines:
    line = line.strip()
    clean_lines.append(line)
to_be_corrected = clean_lines[::3]
correct_answers = clean_lines[1::3]
print(len(to_be_corrected))


1008


In [25]:
new_sents = []
for sent in to_be_corrected:
    sent = correct_sentence(sent)
    sentence = sent + '\n'
    new_sents.append(sentence)
print(new_sents[:10])

['ну вот собственно и день весь вышел\n', 'коридоры казались огромными было довольно пусто мы бродили по лестницам этажам\n', 'а так все принципе нормально\n', 'надеюсь порисовать но посмотрю какая бдеть погода\n', 'хотела посчитать скока я за седня выпила воды\n', 'приметы прыщи на бороде\n', 'сейчас она победила на конкурсе русского языка в японии и получила бесплатный билет в россию\n', 'впрочем официальные портреты не всегда высокого качества увы востребованы и другими менее именитыми особами\n', 'оказывается можно быть счастливой и несчастной одновременно\n', 'сегодня яичницей никто не завтракал как впрочем и вчера на ближайшем к нам рынке мы ели фруктовый салат со свежеотжатым соком как в старые добрые времена в бразилии\n']


In [26]:
fl = open('corrected_3.txt', 'w')
fl.write(''.join(new_sents)) 
fl.close() 

In [27]:
fl = open('to_be_corrected.txt', 'w')
fl.write('\n'.join(to_be_corrected)) 
fl.close() 

In [28]:
fl = open('correct_answers.txt', 'w')
fl.write('\n'.join(correct_answers)) 
fl.close() 

In [29]:
index_list = list(range(len(to_be_corrected)))
indexes = ''
for i in index_list:
    indexes += str(i)
    indexes += '\n'


In [30]:
fl_1 = open('index_file.txt', 'w')
fl_1.write(indexes)
fl_1.close() 

### Результаты
Precision=40.78
Recall=38.02
FMeasure=39.36
Accuracy=47.22 <br>
354 868 931